In [1]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC,SVR

# Read Data

In [15]:
# df=pd.read_csv('INTC_indicators_final.csv')
# df=pd.read_csv('MSFT_indicators_final.csv')
df=pd.read_csv('NVDA_indicators_final.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume,H-L,O-C,7 DAYS MA,...,10 DAYS EMA,20 DAYS EMA,RSI,MACD,H-O,O-L,6 DAYS PSY,24 DAYS AR,5 DAYS BR,6 DAYS WMS
0,1999-01-22,1.750000,1.953125,1.552083,1.640625,1.508857,67867200.0,0.401042,-0.109375,0.000000,...,1.640625,1.640625,NaN,0.000000,0.203125,0.197917,0.000000,0.000000,0.000000,0.000000
1,1999-01-25,1.770833,1.833333,1.640625,1.812500,1.666928,12762000.0,0.192708,0.041667,0.000000,...,1.671875,1.656994,NaN,0.013711,0.062500,0.130208,0.000000,0.000000,0.000000,0.000000
2,1999-01-26,1.833333,1.869792,1.645833,1.671875,1.537597,8580000.0,0.223958,-0.161458,0.000000,...,1.671875,1.658411,NaN,0.013079,0.036458,0.187500,0.000000,0.000000,0.000000,0.000000
3,1999-01-27,1.677083,1.718750,1.583333,1.666667,1.532807,6109200.0,0.135417,-0.010417,0.000000,...,1.670928,1.659198,NaN,0.012019,0.041667,0.093750,0.000000,0.000000,0.000000,0.000000
4,1999-01-28,1.666667,1.677083,1.651042,1.661458,1.528017,5688000.0,0.026042,-0.005208,0.000000,...,1.669206,1.659413,NaN,0.010636,0.010417,0.015625,0.000000,0.000000,1.134616,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5491,2020-11-13,545.090027,546.750000,522.369995,531.880005,531.880005,8657800.0,24.380005,-13.210022,547.381426,...,536.543296,536.106889,49.028108,1.627440,1.659973,22.720032,50.000000,0.611781,0.584727,67.587531
5492,2020-11-16,526.900024,545.979980,526.239990,540.609985,540.609985,10344400.0,19.739990,13.709961,544.539995,...,537.282694,536.535755,51.183731,1.831262,19.079956,0.660034,50.000000,0.694174,1.060422,57.009579
5493,2020-11-17,542.099976,543.849976,532.150024,536.890015,536.890015,7800700.0,11.699951,-5.209961,540.855704,...,537.211298,536.569494,50.905753,1.673332,1.750000,9.949951,50.000000,0.701051,2.461601,30.213133
5494,2020-11-18,538.609985,544.229980,527.400024,537.150024,537.150024,12773100.0,16.829956,-1.459961,534.342852,...,537.200157,536.624783,56.779918,1.551271,5.619995,11.209961,66.666667,0.670607,1.080560,41.241153


In [16]:
# # preprocessing : price rise (1) or fall (-1) label - compare to last day
# def add_label(df, column):
#     label = []
#     for i in range(len(df)):
#         if(i == 0):
#             label.append(-1)
#             continue
#         if(df[column][i] >= df[column][i - 1]):
#             label.append(1)
#         else:
#             label.append(-1)
#     return pd.Series(label)
    

# df["label"] = add_label(df, "Close")
# df

# preprocessing : price rise (1) or fall (-1) label compare to the next day

In [17]:
def add_label(df, column):
    label = []
    for i in range(len(df)):
        if(i==len(df)-1):
            label.append(0)
            continue
        if(df[column][i+1] >= df[column][i]):
            label.append(1)
        else:
            label.append(0)
    return pd.Series(label)
    

df["label"] = add_label(df, "Close")
df

,Date,Open,High,Low,Close,Adj Close,Volume,H-L,O-C,7 DAYS MA,...,20 DAYS EMA,RSI,MACD,H-O,O-L,6 DAYS PSY,24 DAYS AR,5 DAYS BR,6 DAYS WMS,label
0,1999-01-22,1.750000,1.953125,1.552083,1.640625,1.508857,67867200.0,0.401042,-0.109375,0.000000,...,1.640625,NaN,0.000000,0.203125,0.197917,0.000000,0.000000,0.000000,0.000000,1
1,1999-01-25,1.770833,1.833333,1.640625,1.812500,1.666928,12762000.0,0.192708,0.041667,0.000000,...,1.656994,NaN,0.013711,0.062500,0.130208,0.000000,0.000000,0.000000,0.000000,0
2,1999-01-26,1.833333,1.869792,1.645833,1.671875,1.537597,8580000.0,0.223958,-0.161458,0.000000,...,1.658411,NaN,0.013079,0.036458,0.187500,0.000000,0.000000,0.000000,0.000000,0
3,1999-01-27,1.677083,1.718750,1.583333,1.666667,1.532807,6109200.0,0.135417,-0.010417,0.000000,...,1.659198,NaN,0.012019,0.041667,0.093750,0.000000,0.000000,0.000000,0.000000,0
4,1999-01-28,1.666667,1.677083,1.651042,1.661458,1.528017,5688000.0,0.026042,-0.005208,0.000000,...,1.659413,NaN,0.010636,0.010417,0.015625,0.000000,0.000000,1.134616,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5491,2020-11-13,545.090027,546.750000,522.369995,531.880005,531.880005,8657800.0,24.380005,-13.210022,547.381426,...,536.106889,49.028108,1.627440,1.659973,22.720032,50.000000,0.611781,0.584727,67.587531,1
5492,2020-11-16,526.900024,545.979980,526.239990,540.609985,540.609985,10344400.0,19.739990,13.709961,544.539995,...,536.535755,51.183731,1.831262,19.079956,0.660034,50.000000,0.694174,1.060422,57.009579,0
5493,2020-11-17,542.099976,543.849976,532.150024,536.890015,536.890015,7800700.0,11.699951,-5.209961,540.855704,...,536.569494,50.905753,1.673332,1.750000,9.949951,50.000000,0.701051,2.461601,30.213133,1
5494,2020-11-18,538.609985,544.229980,527.400024,537.150024,537.150024,12773100.0,16.829956,-1.459961,534.342852,...,536.624783,56.779918,1.551271,5.619995,11.209961,66.666667,0.670607,1.080560,41.241153,1


# Normalize the features

In [19]:
features = ["Open", "High", "Low", "Volume","Close","H-L","O-C","7 DAYS MA","14 DAYS MA","21 DAYS MA","7 DAYS STD DEV","RSI","10 DAYS EMA","20 DAYS EMA","MACD","H-O","O-L","6 DAYS PSY","24 DAYS AR","6 DAYS WMS"
]
autoscaler = StandardScaler()
df[features] = autoscaler.fit_transform(df[features])
df

,Date,Open,High,Low,Close,Adj Close,Volume,H-L,O-C,7 DAYS MA,...,20 DAYS EMA,RSI,MACD,H-O,O-L,6 DAYS PSY,24 DAYS AR,5 DAYS BR,6 DAYS WMS,label
0,1999-01-22,-0.553624,-0.552173,-0.555474,-0.554957,1.508857,4.559184,-0.385771,-0.043622,-0.574246,...,-0.559207,NaN,-0.198945,-0.317797,-0.296195,-2.510778,-2.969901,0.000000,-1.509725,1
1,1999-01-25,-0.553403,-0.553423,-0.554517,-0.553134,1.666928,-0.266253,-0.439512,0.014010,-0.574246,...,-0.559029,NaN,-0.194926,-0.379927,-0.322033,-2.510778,-2.969901,0.000000,-1.509725,0
2,1999-01-26,-0.552740,-0.553042,-0.554461,-0.554626,1.537597,-0.632461,-0.431451,-0.063495,-0.574246,...,-0.559013,NaN,-0.195111,-0.391433,-0.300170,-2.510778,-2.969901,0.000000,-1.509725,0
3,1999-01-27,-0.554398,-0.554618,-0.555136,-0.554681,1.532807,-0.848824,-0.454291,-0.005863,-0.574246,...,-0.559005,NaN,-0.195422,-0.389132,-0.335946,-2.510778,-2.969901,0.000000,-1.509725,0
4,1999-01-28,-0.554508,-0.555053,-0.554405,-0.554736,1.528017,-0.885707,-0.482505,-0.003876,-0.574246,...,-0.559003,NaN,-0.195827,-0.402938,-0.365759,-2.510778,-2.969901,1.134616,-1.509725,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5491,2020-11-13,5.209741,5.133139,5.071121,5.070664,531.880005,-0.625649,5.799806,-5.042319,5.289984,...,5.258370,-0.247109,0.278104,0.325858,8.298437,-0.055835,-1.197408,0.584727,0.680369,1
5492,2020-11-16,5.016794,5.125103,5.112930,5.163285,540.609985,-0.477957,4.602875,5.229299,5.259543,...,5.263038,-0.122229,0.337850,8.022236,-0.119847,-0.055835,-0.958694,1.060422,0.337603,0
5493,2020-11-17,5.178025,5.102875,5.176779,5.123818,536.890015,-0.700703,2.528879,-1.989807,5.220072,...,5.263405,-0.138333,0.291557,0.365633,3.425265,-0.055835,-0.938770,2.461601,-0.530704,1
5494,2020-11-18,5.141005,5.106841,5.125462,5.126576,537.150024,-0.265281,3.852207,-0.558953,5.150298,...,5.264007,0.201972,0.255777,2.075447,3.906095,0.762480,-1.026974,1.080560,-0.173354,1


# Select the technical indicators only

In [20]:
X = df.drop(['label','Date','5 DAYS BR','Adj Close','Open','High','Low','Close','Volume','H-L','O-C','H-O','O-L'], axis=1)
y=df['label']
print("Dimension of the Data Matrix X: ", X.shape)
print("Dimension of the Target: ", y.shape)

Dimension of the Data Matrix X:  (5496, 11)
Dimension of the Target:  (5496,)


# Split data into train and test

In [21]:
# start_time, split_time, end_time=2000, 7500,8000 #MSFT #INTC
start_time, split_time, end_time=1000, 4000,5200 #NVDA
X_train,X_test=X.loc[start_time:split_time],X.loc[split_time:end_time]
y_train,y_test=y.loc[start_time:split_time],y.loc[split_time:end_time]

In [22]:
X_train

,7 DAYS MA,14 DAYS MA,21 DAYS MA,7 DAYS STD DEV,10 DAYS EMA,20 DAYS EMA,RSI,MACD,6 DAYS PSY,24 DAYS AR,6 DAYS WMS
1000,-0.529536,-0.531603,-0.531795,-0.459938,-0.530079,-0.531058,-0.498956,-0.231456,-0.055835,-0.763751,0.904678
1001,-0.529510,-0.531745,-0.532041,-0.459860,-0.530363,-0.531300,-0.498956,-0.232811,-0.055835,-1.052554,1.422050
1002,-0.530189,-0.531917,-0.532259,-0.470850,-0.531122,-0.531799,-0.557674,-0.239800,-0.055835,-1.174268,1.463409
1003,-0.531260,-0.532239,-0.532752,-0.468765,-0.532017,-0.532395,-0.807632,-0.248083,-0.874149,-1.092495,1.536788
1004,-0.531872,-0.532548,-0.533314,-0.455475,-0.532925,-0.533028,-0.788047,-0.256118,-1.692464,-1.049995,1.691773
...,...,...,...,...,...,...,...,...,...,...,...
3996,-0.350431,-0.353739,-0.356337,-0.421380,-0.351660,-0.354942,1.061788,-0.060177,-0.055835,-0.391551,0.147293
3997,-0.350615,-0.352890,-0.356036,-0.441396,-0.351707,-0.354607,0.823934,-0.069458,-0.055835,-0.430725,0.383535
3998,-0.350982,-0.352458,-0.355548,-0.457769,-0.352643,-0.354780,0.401205,-0.088962,-0.874149,-0.463159,1.665848
3999,-0.351456,-0.352257,-0.355419,-0.438352,-0.353428,-0.354948,0.056933,-0.105730,-1.692464,-0.365069,1.604814


# Find optimal hyperparameter

In [24]:
%%time

warnings.filterwarnings('ignore')
param_grid = {'kernel': ['linear'],'C': [0.1,1, 10, 100, 1000]}
clf = SVC(kernel="linear")

clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
clf.fit(X_train, y_train)

params_optimal = clf.best_params_

print("Best Score (accuracy): %f" % clf.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal)
print("\n")

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   51.5s finished


Best Score (accuracy): 0.510494
Optimal Hyperparameter Values:  {'C': 100, 'kernel': 'linear'}


Wall time: 53.7 s


# Run SVC()

In [25]:
%%time
svm = SVC(**params_optimal)
svm.fit(X_train, y_train)
pred_y = svm.predict(X_test)



Wall time: 2.16 s


In [28]:
y_test_predicted = pred_y


accuracy_score_test = np.mean(y_test_predicted == y_test)
print("\nAccuracy: ", accuracy_score_test)

print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))

precision_test = precision_score(y_test, y_test_predicted, average='micro') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, y_test_predicted, average='micro')
print("\nTest Recall = %f" % recall_test)


f1_test = f1_score(y_test, y_test_predicted, average='micro')
print("\nTest F1 Score = %f" % f1_test)


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))


Accuracy:  0.4929225645295587

Test Confusion Matrix:
[[358 189]
 [420 234]]

Test Precision = 0.492923

Test Recall = 0.492923

Test F1 Score = 0.492923

Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.65      0.54       547
           1       0.55      0.36      0.43       654

    accuracy                           0.49      1201
   macro avg       0.51      0.51      0.49      1201
weighted avg       0.51      0.49      0.48      1201



# Run SVM-RFE

In [29]:
svc = SVC(**params_optimal)
rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
rfe.fit(X_train, y_train)

RFE(estimator=SVC(C=100, kernel='linear'), n_features_to_select=1)

In [30]:
print(rfe.ranking_)
print(rfe.n_features_)
print(rfe.support_)

[ 6  2  5  4  3  1 10  7  8 11  9]
1
[False False False False False  True False False False False False]


In [31]:
pred_y_rfe = rfe.predict(X_test)

In [32]:
print(pred_y_rfe)

[1 1 1 ... 1 1 1]


In [33]:
y_test_predicted = pred_y_rfe


accuracy_score_test = np.mean(y_test_predicted == y_test)
print("\nAccuracy: ", accuracy_score_test)

print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))

precision_test = precision_score(y_test, y_test_predicted, average='micro') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, y_test_predicted, average='micro')
print("\nTest Recall = %f" % recall_test)


f1_test = f1_score(y_test, y_test_predicted, average='micro')
print("\nTest F1 Score = %f" % f1_test)


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))


Accuracy:  0.5445462114904246

Test Confusion Matrix:
[[  0 547]
 [  0 654]]

Test Precision = 0.544546

Test Recall = 0.544546

Test F1 Score = 0.544546

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       547
           1       0.54      1.00      0.71       654

    accuracy                           0.54      1201
   macro avg       0.27      0.50      0.35      1201
weighted avg       0.30      0.54      0.38      1201



In [34]:
pred_y_train = rfe.predict(X_train)

In [35]:
print(pred_y_train)

[1 1 1 ... 1 1 1]


In [ ]:
y_train_predicted = pred_y_train


accuracy_score_test = np.mean(y_train_predicted == y_train)
print("\nAccuracy: ", accuracy_score_test)

print("\nTest Confusion Matrix:")
print(confusion_matrix(y_train, y_train_predicted))

precision_test = precision_score(y_train, y_train_predicted, average='micro') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_train, y_train_predicted, average='micro')
print("\nTest Recall = %f" % recall_test)


f1_test = f1_score(y_train, y_train_predicted, average='micro')
print("\nTest F1 Score = %f" % f1_test)


print("\nClassification Report:")
print(classification_report(y_train, y_train_predicted))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

# Create the RFE object and compute a cross-validated score.
svc = SVC(**params_optimal)
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=svc, step=1, cv=2)
rfecv.fit(X_train, y_train)
print("Optimal number of features : %d" % rfecv.n_features_)

In [ ]:
print(rfecv.ranking_)
print(rfecv.support_)

In [ ]:
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
pred_y_rfecv = rfecv.predict(X_test)

In [ ]:
print(pred_y_rfecv)

In [ ]:
y_test_predicted = pred_y_rfecv


accuracy_score_test = np.mean(y_test_predicted == y_test)
print("\nAccuracy: ", accuracy_score_test)

print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))

precision_test = precision_score(y_test, y_test_predicted, average='micro') 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, y_test_predicted, average='micro')
print("\nTest Recall = %f" % recall_test)


f1_test = f1_score(y_test, y_test_predicted, average='micro')
print("\nTest F1 Score = %f" % f1_test)


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))